# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logConsulting').getOrCreate()

In [3]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [10]:
df.groupBy('churn').count().show()

+-----+-----+
|churn|count|
+-----+-----+
|    1|  150|
|    0|  750|
+-----+-----+



In [16]:
from pyspark.sql.functions import countDistinct

In [17]:
df.agg(countDistinct('Names')).alias('count').show()

+---------------------+
|count(DISTINCT Names)|
+---------------------+
|                  899|
+---------------------+



In [20]:
df.agg(countDistinct('Location')).alias('count').show()

+------------------------+
|count(DISTINCT Location)|
+------------------------+
|                     900|
+------------------------+



In [21]:
df.agg(countDistinct('Company')).alias('count').show()

+-----------------------+
|count(DISTINCT Company)|
+-----------------------+
|                    873|
+-----------------------+



In [22]:
# Names, Location and Company seem to be irrelevant, since from 900 samples there are hardly any repeated values.

In [24]:
from pyspark.sql.functions import min, max

In [29]:
df.agg({'Onboard_date': 'max'}).show()

+-------------------+
|  max(Onboard_date)|
+-------------------+
|2016-12-28 04:07:38|
+-------------------+



In [30]:
df.agg({'Onboard_date': 'min'}).show()

+-------------------+
|  min(Onboard_date)|
+-------------------+
|2006-01-02 04:16:13|
+-------------------+



In [ ]:
# Onboard_data could be usefult along with other variables, but by itself doesn't explain much.

In [38]:
df.corr('Age', 'Churn')

0.08592553706723399

In [39]:
# Age can be included in a future improve step

In [41]:
df.groupBy(['churn', 'Account_Manager']).count().show()

+-----+---------------+-----+
|churn|Account_Manager|count|
+-----+---------------+-----+
|    1|              0|   66|
|    1|              1|   84|
|    0|              0|  401|
|    0|              1|  349|
+-----+---------------+-----+



In [42]:
df.corr('Account_Manager', 'Churn')

0.07061077173214911

In [43]:
# Effectively, random account manager doesn't seem to influence in the customer churn decision

In [23]:
df.summary().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [33]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Model

In [95]:
my_cols = ['Age', 'Total_Purchase', 'Years', 'Num_Sites']

In [96]:
my_cols_selected = df.select(my_cols)

In [97]:
my_cols_selected.na.drop().summary().show()

+-------+-----------------+-----------------+-----------------+------------------+
|summary|              Age|   Total_Purchase|            Years|         Num_Sites|
+-------+-----------------+-----------------+-----------------+------------------+
|  count|              900|              900|              900|               900|
|   mean|41.81666666666667|10062.82403333334| 5.27315555555555| 8.587777777777777|
| stddev|6.127560416916251|2408.644531858096|1.274449013194616|1.7648355920350969|
|    min|             22.0|            100.0|              1.0|               3.0|
|    25%|             38.0|          8480.93|             4.45|               7.0|
|    50%|             42.0|         10041.13|             5.21|               8.0|
|    75%|             46.0|         11758.69|             6.11|              10.0|
|    max|             65.0|         18026.01|             9.15|              14.0|
+-------+-----------------+-----------------+-----------------+------------------+



In [98]:
# There aren't null values in this columns

In [99]:
my_cols = ['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'churn']

In [100]:
my_data = df.select(my_cols)

In [101]:
from pyspark.ml.feature import VectorAssembler

In [102]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites'], 
                          outputCol='features')

In [103]:
from pyspark.ml.classification import LogisticRegression

In [104]:
log_reg_model = LogisticRegression(featuresCol='features', labelCol='churn')

In [105]:
from pyspark.ml import Pipeline

In [106]:
pipeline = Pipeline(stages=[assembler, log_reg_model])

In [107]:
train_data, test_data = my_data.randomSplit([0.7, 0.3])

In [108]:
fit_model = pipeline.fit(train_data)

In [109]:
results = fit_model.transform(test_data)

In [110]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [111]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [112]:
results.select('churn', 'prediction').show()

+-----+----------+
|churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
+-----+----------+
only showing top 20 rows



In [113]:
AUC = my_eval.evaluate(results)

In [114]:
AUC

0.7963800904977375

### Predict on new data

In [115]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True,
                              header=True)

In [116]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [117]:
final_results = fit_model.transform(new_customers)

In [120]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

